# Python (EPAM, 2020), lecture 08

## Python object model

Full documentation
https://docs.python.org/3/reference/datamodel.html


# Section 1. Everything is object

  three main characteristics of each object:
   - type (or class)
   - identity (id)
   - value


### Objects, references and values. Mutability and immutability


```python
x = 1
type(x)  # int
id(x)    # address
x

1
type(1)   # int
id(1)     # address same as x
x is 1    # true

y = 1     
type(y)   # int
id(y)     # same as x and 1
x is y    # true !

y = 2
id(y)     # different!
```

You see that objects (data) and variables (syntaxic constructs) are actually decoupled from each other, and _all_ data in python is accessed through variables _by reference_.


Consider the following example:

In [30]:
x = {'a': 1}
y = x
x is y  # should be True
y['b'] = 2
x

{'a': 1, 'b': 2}

When assigning to variable, you are not copying data, you are passing a reference to a variable to the very same object. In order to create a new copy of data in our case we should've used 

```python
y = dict(x)
# or y = x.copy()
# or y = copy.copy(x)
# or y = copy.deepcopy(x)
```
what's the difference? Here's a example. You know that some types may be _mutable_ and some may be _immutable_. For example, integer values are immutable. Tuples are immutable. Sets are immutable. BUT, those immutable objects (like collections), can have arbitrary sets of items, and those on their own can be mutable.

```python
x = ('test', {'me': 1})
x[1]
x[2] = 'something'  # error
x[1]['test'] = 'abc'  # will work
```

So, when you perform a simple .copy(), or _shallow_ copy, only top level contents are copied. _Deep copy_ instead will try to (nestedly) dublicate also collection contents.


dont forget basic immutable types: tuples, sets, strings, bytes.

## Type


Each object has a _type_. (actually, types are also objects and they are an instance of their own, but that''s a little bit of advanced thing right now). 

Examples of types:
   
   - built-in types (int, bool, str, ...)
   - collections (list, set, dict, tuple, ...)
   - classes
   - function types (objects that you can call() )

## Object creation and initialization


when you are using object somewhere (like assigning to variable), actual data (an object) has to be created. This holds true for all objects (but some commonly-used objects, like True, False, some integers are actually pre-created).  To create an object, a _constructor_ has to be called.




```python
class C:
    pass

x = C()
```


Under the hood, there 2 actual things happened. First, a ``C.__new__`` was called that is used to create a new instance of class C - an actual object. This is a _static_ method of C (means it just a simple function in C namespace, that is required to take class _type_ as argument, C in this case). After that, an  _instance_ method of newly created _object_ (with type C) ``__init__`` is called with same arguments (save for the first one).



Reminder of difference between static methods, class methods and instance methods.


```python
class C():
    @classmethod
    def hello_class(cls):
        print(f'{str(cls)} welcomes you')

    @staticmethod
    def hello_static():
        print(f'hello from {str(C)}!')
        
    def hello_instance(self):
        print(f'Hello from {self.__class__)')
```

- static method - just a plain function that is called from C namespace (we'll talk about namespaces later)
- class method - a function that takes _current class_ as an argument
- instance method - a function that operates on current instance.


## What else is an object?


 - types
 - modules
 - functions
 - classes
 - everything


some esotheric example:

```python

def z(a):
   print(a)

x = type(z)  # type `function`
y = x(z.__code__, globals())  # create a new instance of function with same code as z, same global variables
y('test')   # run it
type(x)  # type `type`
```

``type`` is a bit special, it is an instance of its own.

# Section 2: Protocols and dunder methods


There's a very neat and useful built-in function that allows you to see what is actually inside of an object, dir().


You will see some:

    - methods (hello)
    - attributes (name)
    - __double under (dunder) methods__ and attributes


those dunder methods/attributes are actually powering all that neat and cozy python syntax (and you will be able to use that to your advantage). Most of them are inherited from class `object`.


In [31]:
class C:
    def __init__(self, name):
        self.name = name

    def hello(self):
        print('test')

x = C('superobject')
dir(x)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'hello',
 'name']

some of standard dunder methods ones that you will use:

```
  - x.__class__ - reference to a current instance class
  - x.__doc__ - docstring
  - x.__dict__ - dictionary containing _instance_ attributes (the ones that were defined on class go into x.__class__.__dict__ . It is this object `namespace`
  - x.__str__ - controls string conversion of X (used by string formatting or printing).
  - x.__repr__ - controls string representation (that could be used to reconstruct object, but nobody uses that anyway, so it's analogue to __str__ usually)
  - x.__init__ , x.__new__ - we already mentioned those
  - x.__hash__ - to be used in dicts (hashmaps)
  - x.__bool__ - how it evaluates to boolean, for example in if statements.
  - x.__eq__, x.__lt__, x.__gt__, ... - comparison and equality.
```  


If you (re)define those dunder methods, you can modify object behaviour, how it fits into your programming code. This allows flexibility of creating a specific object exactly for your need.

For example, you could create class to search and instantly enumerate lines from the wikipedia article by implementing dunder methods `__iter__` and `__next__` (from iterator protocol):

```python
class WikiReader():
    def __init__(self, search):
        self.line_no = 0
        self.lines = requests.get('https://wikipedia.org/wiki/{search}').text
      
    def __iter__(self):
        return self
          
    def __next__(self):
        if self.line_no > len(self.lines):
            raise StopIteration
        to_return = self.article.splitlines()[self.line_no]
        self.line_no += 1

      

for line in WikiReader(search='python'):
    if 'programming language' in line.lower():
        print('Success!')
        break
```            




Or, by implementing `__getattr__` and `__setattr__` you could extend simple dictionary class to be able to access dictionary elements as attributes:

```python
class MagicDict(dict):
    pass   # not going to implement

x = MagicDict({'a':1, 'b': 'test'})
x.b   # now allowed!
x.c = 'another collection item'
```

This kind of flexibility is supported by _protocols_ - conventions on what dunder method does what and what is needed to be implemented for specific language feature.
In other languages those may be called _interfaces_, but in python it's way less formal.

List of protocols that you will encounter that allow to use object in certain language constructs:
```
   - Iteration:
     * Iterable (__iter__)
     * Iterator  (__iter__, __next__)
   - Collections (with __getitem__ inherited from object)
     * Sized (__len__)
     * Sequence (__len__)
     * Container (__contains__)
     * Collection (__len__, __iter__, __contains__)
   - Context managers 
     * synchronous (__enter__, __exit__)
     * asynchronous (__aenter__, __aexit__)
   - Callable  (__call__)
   - Asynchronous programming
     * awaitable (__await__)
     * asynchronous iterable (__aiter__)
     * asynchronous iterator (__anext__, __aiter__)
```     

Additionally, there are protocols related to object elements access:

```    
  - to access attributes and methods through . notation
      __setattr__, __getattr__, __getattribute__, __setattribute__ 
  - to access collection _items_ through ['key'] notation:
      __getitem__, __setitem__, __delitem__, __missing__
```        


Type conversion and comparison:
```__bool__, __int__, __float__, __str__, __lt__, __eq__, etc.```

Additionally, there are additional conventions that are commonly used.
  - File-like object (.read(), .write(), see io module)

There are probably more of them and more will follow with language development.


# Section 3: Object dissection

In [32]:
class C():
    def __init__(self, name):
        self.name = name

    def hello(self):
        print('test')

    @classmethod
    def foo(cls):
        pass
    
    print('test')

x = C('abc')
x.hello


test


<bound method C.hello of <__main__.C object at 0x7fc5000711f0>>

In [33]:
x.name


'abc'

In [34]:
x.foo


<bound method C.foo of <class '__main__.C'>>

In [35]:
x.__dict__ 

{'name': 'abc'}

In [36]:
x.__class__.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.C.__init__(self, name)>,
              'hello': <function __main__.C.hello(self)>,
              'foo': <classmethod at 0x7fc500071ee0>,
              '__dict__': <attribute '__dict__' of 'C' objects>,
              '__weakref__': <attribute '__weakref__' of 'C' objects>,
              '__doc__': None})

everything that you access through dot (.) is object attribute. everything you access through brackets is collection item.
When you are accessing an attribute of object, first the lookup is being done on object namespace.
If object is not found, it is being looked up in ``x.__class__``, and from that point a lookup is made through parent classes according to class method resolution order (``x.__class__.__mro__``).

This behaviour can be changed with altering ``__getattr__, __setattr__, __getattribute__, __setattribute__``  dunder methods.

You also probably noticed that x.hello and x.foo are ALSO accessed as attributes. How they operate is a bit complex topic, organically they seem as functions _bounded_ to class or object instances. Note that they dont appear in `x.__dict__`, but appear in `C.__dict__`. This will be explained in next section.

Since you may change object attributes on the fly, it means that you will be able to create or modify methods on object instance on the fly.

# DOWN THE RABBIT HOLE

Advanced stuff

# Section 4: properties, attributes and descriptor protocol


So, what happens when we access some attributes of a class or instance?
We already figured out that a lookup in ``__dict__`` being done.

It's interesting to see how things are organized. 

In [37]:
class C():
    def f(self, a):
        print(a)

c = C()

print(c.f)   # bound method C.f of <__main__.C object at 0x7fcbfefa9fd0>
print(c.__dict__)   # but no F function here
print(c.__class__.__dict__['f'])   # <function C.f at 0x7fcbfef6c4c0>


<bound method C.f of <__main__.C object at 0x7fc5001622e0>>
{}
<function C.f at 0x7fc5001919d0>


So, function f is located in class C namespace, but not in instance namespace. But function inside class namespace is unbounded to instance. When you think of that c.f is, first idea is that it theoretically should be
equivalent to attribute function with bound variable self to object instance AND be located in instance namespace.
But it's not there. To understand what trick python does to make it works we should look at another example (class property).


In [38]:
class X():
   @property
   def f(self):
      return 10

x = X()
print(x.f)   # 10
print(x.__dict__)  # empty
print(x.__class__.__dict__['f'])  # <property object at 0x7fcbfe736f90>


10
{}


BINGO. This is not a function, but different kind of object (property), but that behaves the same way related to instance binding. Maybe they implement some sort of protocol that converts them  so they would be accessible by concrete instances and instances of class successors. 


A quote from documentation::

    A class instance is created by calling a class object (see above). A class instance has a namespace implemented as a dictionary which is the first place in which attribute references are searched. When an attribute is not found there, and the instance’s class has an attribute by that name, the search continues with the class attributes. If a class attribute is found that is a user-defined function object, it is transformed into an instance method object whose __self__ attribute is the instance.


So that's when all the magic happens. Something is transformed into something. How and why they are transformed?
The answer is this:

when object is looked up in owner namespace, if that object is simple, it's just returned
BUT if it implements ANY of the methods `__get__`, `__set__` or `__delete__`, the behaviour is altered.
Those methods specify ``descriptor protocol``.

a = A()
a.b

3 rules:
   - `A.__dict__['b']` has no `__get__`. -> return value of b as is (if `a.__dict__` does have 'b' then it is returned instead)
   - `A.__dict__['b']` has `__delete__` or `__set__` -> data descriptor
   - `A.__dict__['b']` has `__get__` -> non-data descriptor, a.b is equivalent to  `A.__dict__['b'].__get__(a, A)`


In [39]:
# Lets see how descriptor calls transform simple functions to bound instances of object methods:

class C:
  def f(self, a):
     print(a)

x = C()
x.f
# <bound method C.f of <__main__.C object at 0x7f7687ff3fd0>>
C.f
# <function C.f at 0x7f7687fb64c0>
C.f.__get__
#<method-wrapper '__get__' of function object at 0x7f7687fb64c0>
def f(self, a):
  print(a)
 
f.__get__
# <method-wrapper '__get__' of function object at 0x7f7687d7a4c0>
f.__get__(x)
#<bound method f of <__main__.C object at 0x7f7687ff3fd0>>


<bound method f of <__main__.C object at 0x7fc500196a90>>

Why the complexity?

Because it allows to implement inheritance and accessing methods and properties of superclasses.
When you were busy with OOP, you invoked superclasses methods like this

```python
class C():
   def f(self, a):
      print('inside C')
      print(a)


class Inherited(C):
   def f(self, a):
       print('before super call')
       super().f(a)
       print('after super call')
       print(a)

x = Inherited()
x.f(1)
```



To understand it, we need to understand what super() returns. super() cannot return current instance, and it will not create another instance of superclass.
Instead, it creates a special object that will route calls of its attributes to descriptors of superclass of x.



In [43]:
# illustrates common super() calls
class C():
   def f(self):
       return (
           super(),    # ancestor, bound to object instance
           super(C),   # ancestor, unbound to object instance
           super(C, self)  # same as call #1
       )  # we will see what different calls to super may return
 
x = C()
z = x.f()
z


(<super: __main__.C, <__main__.C at 0x7fc500071520>>,
 <super: __main__.C, None>,
 <super: __main__.C, <__main__.C at 0x7fc500071520>>)

super(C, self) or plain super() object implements `__getattribute__` -> the method we use to access attributes through dots (.)
and this `__getattribute__`
  - will go through C.__mro__ starting from after C.
  - looks up requested attributes or descriptors and return them.

Programming with descriptors can make some very interesting practical cases, but in general it's not advised to use them directly.

Esoteric example: Двое из ларца

## types module and modifying objects on the fly

Now that you know how things work, you can dynamically modify all possible classes, instances, etc by adding or removing methods.
It does not mean you SHOULD do that, but you CAN. It allows you to design some creative things, like object instance that has dynamically changing
methods, attributes and properties. In most cases it's a bad idea to design your code around that, but in rare cases it will help you.
I can think of example when you have unpickled object that you need to perform additional functionality, you may add a method on the fly.
Generally it's a huge hack and a bad idea.

Module `types` will help you code a multitude of those bizarre property and method objects without implementing everything yourself.

In [41]:
#Example: adding instance method

class C:
  pass

x = C()
import types
def f(self, a):
  print(self, a)

x.f = f
# x.f('1') # will not work
x.f = types.MethodType(f, x)
x.f('a')
# <__main__.C object at 0x7f285180afd0> a
# YOU CAN ALSO "STEAL" METHODS FROM OTHER CLASSES WITH THE SAME TECHNIQUE


<__main__.C object at 0x7fc500082d00> a


# Section 5: Metaclasses

`What is metaclasses?` is one of the most popular question on interviews.
A little back ago I talked that everything is an object in python, including classes.

```
class C:
   pass

C  # <class '__main__.C'>
type(C)  # <class "type" >

Metaclass is TYPE that creates instance of CLASS objects.

A = type('A', C.__bases__, dict(C.__dict__))
A  # <class '__main__.C'>
type(A)  # <class "type" >
```

- Metaclasses construct the whole instance of class when invoked
- exact steps:
   - creates a separate namespace
   - executes class body code in that namespace, it populates the namespace
   - creates instance of class object. class namespace becomes class.__dict__
- We can define our own metaclasses or derive them from type. 
- Since abovementioned steps are somewhat complicated, usually metaclasses are implemented as subclasses of type.
- Since we are now constructing class object ourselves, we have FULL CONTROL on how it will be created.
    - you can perform validation of function names. ->> module abc implements abstract classes.
    - you can add or remove additional class, static or instance methods automatically.
    - do whatever you want. This is one of the most powerful language feature in python.


In [42]:
#How it looks in code:

class A:
    pass

class B:
    pass


class MyMeta(type):   # is a metaclass
    def __new__(cls, name, bases, dct):
        cls_instance = super().__new__(cls, name, bases, dct)
        def think(self):
            print('Cogito ergo sum')
        cls_instance.think = think   # adding instance method to ALL classes created with this metaclass
        return cls_instance

class C(A, B,  metaclass=MyMeta):    # A and B are BASES, using custom metaclass MyMeta
    pass

x = C()
x.think()


Cogito ergo sum


One of examples that is possible with using metaclasses is creating an ORM class object.
Say, you have some tables in database. Then you specify

```
class Author(metaclass=ORM):
    pass

or 

class Book(metaclass=ORM):
    __table__ = 'books'
```    

and ORM metaclass will connect to database, check if there's table name `author` or `books`, create attributes on this class that correspond to this table fields,
add capability to query database to the class, etc. Pure magic.